In [1]:
import re
import xml.etree.cElementTree as ET
from collections import defaultdict
import pprint

osm_path = "C:\Users\Leo\Anaconda3\P3 Project OSM\singapore.osm"
post_re = re.compile(r'\d{6}')

def check_pc(osmfile):
    '''Checks if postal codes matches regular expression.
        Adds attributes which do not match to list and returns it.'''
    osm_file = open(osm_path, "r")
    pc_codes = []
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == 'addr:postcode' and not post_re.search(tag.attrib['v']):
                    pc_codes.append(tag.attrib['v'])
    osm_file.close()
    return pc_codes

pprint.pprint(check_pc(osm_path))

['29425',
 '29426',
 '29427',
 '29464',
 '29466',
 '29432',
 '135',
 '29463',
 '29461',
 '29461',
 '29464',
 '29433',
 '81300',
 '81300',
 '81300',
 '81300',
 '81200',
 '29464',
 '29464',
 '80200',
 '81620',
 '81310',
 '81310',
 '81310',
 '29464',
 '81300',
 '81300',
 '81300',
 '81300',
 '81300',
 '81300',
 '81300',
 '81300',
 '81300',
 '81300',
 '81300',
 '81300',
 '81300',
 '81300',
 '80100',
 '81300',
 '81300',
 '81300',
 '81300',
 '81300',
 '81200',
 '81300',
 '81200',
 '29432',
 '80000',
 '81200',
 '81200',
 '80400',
 '80400',
 '80400',
 '2424',
 '81750',
 '81750',
 '81750',
 '81750',
 '81750',
 '81750',
 '29427',
 '80400',
 '81100',
 '80050',
 '81800',
 '81800',
 '81800',
 '81800',
 '81800',
 '81800',
 '81800',
 '81800',
 '80400',
 '81200',
 '80300',
 '80250',
 '29427',
 '81100',
 '80150',
 '80150',
 '80150',
 '80000',
 '05901',
 '81550',
 '81100',
 '81100',
 '81300',
 '79100',
 '80250',
 '80200',
 '80200',
 '81200',
 '81200',
 '81200',
 '81200',
 '81200',
 '80350',
 '81300',
 '8

- Users will not have to worry about submitting wrong data or, remembering the format
- People working or using the data will not have to worry about consistency or errors

Anticipated Problems:
- Standardizing of postal code formats which differs country to country, localized help will be necessary
- Depending on the number of concurrent submissions, the performance of the application/backend may be affected

### Map Area:

Singapore, Singapore

It will be interesting to see what information OSM can tell about my country

Reference Websites:
- Udacity Forums
- http://www.openstreetmap.org/relation/536780#map=11/1.3044/103.8459&layers=G
- https://mapzen.com/data/metro-extracts/metro/singapore/
- https://en.wikipedia.org/wiki/Postal_codes_in_Singapore

### Wrangling Process:
    
- Perform pre-cursory check on downloaded XML file
- Noticed that there were issues with street names and postal codes (More information can be found below)
- Cleaned up abbreviated street names

### Dataset Overview

File Size:
- nodes.csv       110 MB
- nodes_tags.csv  3.55 MB
- ways.csv        12.3 MB
- ways_nodes.csv  41.5 MB
- ways_tags.csv   17 MB

Number of unique contributors: 1753
- cur.execute('''select count(distinct(uid)) from (select uid from nodes union all select uid from ways);''')

Number of nodes: 1402943
- cur.execute('''select count(*) from nodes;''')

Number of ways: 214195
- cur.execute('''select count(*) from ways;''')

Total data points: 1617138

### Data Issues:

After a pre-cursory check and examination of the OSM file, the following ware noticed:
- Abbreviated street names
- Wrong postal code format
- Wrong city entered
- Addresses from beyond Singapore

### Abbreviated Street Names 

Such as dr vs drive, st vs street

While glancing through the OSM file, some of the street names were abbreviated possibly by contributors who were used to shortening common words.

For Singapore, street names may end with numbers such as Bedok North Avenue 3 (My address).
Also depending on how the contributors submit the data, the below code that was used to convert to full street names may not be fully applicable and I may need to amend the code in the future.

In [ ]:
#Update streetnames based on mapping
def update_name(name, mapping):
    for map in mapping:
        if map in name:
            name = re.sub(r'\b' + map + r'\b\.?', mapping[map], name)
    return name

### Wrong Postal Code Format

Such as 'Singapore 408564' or '81200https://www.youtube.com/watch?v=m92AUlOMP2k'

Based on a check, there were a few inconsistent Postal Codes were found.
From the way looked, it seemed like the contributors were unsure of the format to enter them or, they were entered wrongly however, the number affected seems low and can be ignored, removed or, amended.

This was also observed where the addresses were entered instead of the city for the city (key).

### Addresses From Beyond Singapore

A query for cities listed in the dataset showed that most were from Johor Bahru (Malaysia) which is North of Singapore which coincides with the high number of Postal Codes from Johor Bahru.

Possible causes for this could be:

- Extraction boundary limit set by Metro Extracts on non-member accounts which caused Johor Bahru to be included
- Boundary classification errors causing nodes in Malaysia to be set under Singapore
- Tagging errors by contributors when submitting but given the high number, this would be the least likely

In [ ]:
cur.execute('''select a.key, a.value, count(*) as num
from (select * from nodes_tags union all
select * from ways_tags) a
where a.key = 'city'
group by a.key, a.value
order by num desc
limit 20;''')

In [ ]:
['key', 'value', 'num']
[(u'city', u'Johor Bahru', 13836),
 (u'city', u'Singapore', 11302),
 (u'city', u'Batam', 41),
 (u'city', u'SKUDAI', 35),
 (u'city', u'Masai', 13),
 (u'city', u'Ulu Tiram', 10),
 (u'city', u'Iskandar Puteri', 7),
 (u'city', u'Skudai', 7),
 (u'city', u'singapore', 5),
 (u'city', u'Kulai', 4),
 (u'city', u'Nusajaya', 4),
 (u'city', u'Sembawang', 4),
 (u'city', u'Batam Kota', 2),
 (u'city', u'Johor bahru', 2),
 (u'city', u'#01-05', 1),
 (u'city', u'#01-06', 1),
 (u'city', u'#01-33', 1),
 (u'city', u'#01-38/40/42', 1),
 (u'city', u'#01-44', 1),
 (u'city', u'#01-46', 1)]

### Interesting Data

### Top amenities: 
- Top result shows restaurants which fits in perfectly with Singaporeans as we love to eat
- Followed by places of worship, parking lots, cafes, taxi stands, fast food restaurants, atms, banks, toilets and fuel stations
- Delving further on places of worship showed that the top ranking was Muslim however, the results may have be skewed by the data from Johor Bahru. I would expect that the data would be more spread out evenly among the religions

In [ ]:
cur.execute('''select key, value, count(*) as num 
from nodes_tags where key = 'amenity' 
group by key, value 
order by num desc limit 20;''')
            
cur.execute('''select nodes_tags.value, count(*) as num
from nodes_tags
join (select distinct (id) from nodes_tags where value = 'place_of_worship') a
on nodes_tags.id = a.id
where nodes_tags.key = 'religion'
group by nodes_tags.value
order by num desc
limit 20;''')

### Contributors Data

Top contributors are as below:

JaLooNz: 366226 approximately 22.65% of the data
While second place berjaya contributed approximately 7.29% of the data

With the huge disparity between the contributors, I can't help but suspect that automated applications/programs were used to submit the data besides manual means.

OSM does not state which collection and submission methods are allowed but if they were to introduce reward mechanisms, automation may have to be disallowed in-lieu of fairness.

In [ ]:
cur.execute('''select user, count(*) as num
from (select user from nodes union all select user from ways)
group by user
order by num desc
limit 20;''')

In [ ]:
['user', 'num']
[(u'JaLooNz', 366226),
 (u'berjaya', 117874),
 (u'rene78', 80294),
 (u'cboothroyd', 74481),
 (u'Luis36995', 41876),
 (u'ridixcr', 40352),
 (u'kingrollo', 39691),
 (u'lmum', 38867),
 (u'Sihabul Milah', 37310),
 (u'calfarome', 34677),
 (u'jaredc', 26493),
 (u'nikhilprabhakar', 25557),
 (u'Jothirnadh', 24139),
 (u'manings', 22892),
 (u'yurasi', 22361),
 (u'matx17', 21466),
 (u'zomgvivian', 20110),
 (u'poornibadrinath', 19116),
 (u'fusionstream', 18010),
 (u'singastreet', 17642)]

### Conclusion

After exploring the data, it's clear that the information for Singapore incomplete and with errors (Barring the mix of data from Johor Bahru).

The concept of users contributing to the project is intriguing and allows the data to grow much faster than if done only by company staff however, possibilities of errors or format issues will always exist.
I tried to filter and correct the data as much as possible but there were still some error entries left for streets and cities.

A possible suggestion could be to provide users with a format list in the application which they can choose from or, verify the format of the submissions before accepting.

In [53]:
import sqlite3
import csv
import pprint

#Create DB
sqlite_db = 'OSM_Project.sqlite'

#Create connection
conn = sqlite3.connect(sqlite_db)

#Create cursor
cur = conn.cursor()

#Select data
cur.execute('''select a.key, a.value, count(*) as num
from (select * from nodes_tags union all select * from ways_tags) a
where a.key = 'postcode'
group by key, value
order by num desc;''')
for_print = cur.fetchall()

#Close connection
conn.close()

#Set header
header = [tuple[0] for tuple in cur.description]

#Print header and data
pprint.pprint(header)
pprint.pprint(for_print)

['key', 'value', 'num']
[(u'postcode', u'81100', 4123),
 (u'postcode', u'80250', 2250),
 (u'postcode', u'81200', 1671),
 (u'postcode', u'80350', 1564),
 (u'postcode', u'80100', 1354),
 (u'postcode', u'80400', 783),
 (u'postcode', u'80050', 597),
 (u'postcode', u'80300', 532),
 (u'postcode', u'80200', 477),
 (u'postcode', u'80150', 415),
 (u'postcode', u'81310', 63),
 (u'postcode', u'81300', 50),
 (u'postcode', u'<different>', 49),
 (u'postcode', u'069922', 40),
 (u'postcode', u'80000', 28),
 (u'postcode', u'81750', 14),
 (u'postcode', u'129790', 13),
 (u'postcode', u'188979', 12),
 (u'postcode', u'79200', 12),
 (u'postcode', u'81000', 12),
 (u'postcode', u'648886', 10),
 (u'postcode', u'81800', 10),
 (u'postcode', u'018960', 9),
 (u'postcode', u'068897', 9),
 (u'postcode', u'078867', 9),
 (u'postcode', u'018961', 8),
 (u'postcode', u'078878', 8),
 (u'postcode', u'080001', 8),
 (u'postcode', u'120106', 8),
 (u'postcode', u'408832', 8),
 (u'postcode', u'640492', 8),
 (u'postcode', u'1190

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "C:\Users\Leo\Anaconda3\P3 Project OSM\singapore.osm"  # Replace this with your osm file
SAMPLE_FILE = "singapore-sample.osm"

k = 25 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

In [1]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

#Set filepath
OSM_PATH = "C:\Users\Leo\Anaconda3\P3 Project OSM\singapore.osm"

#Set save files
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

#Set regular expressions
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
post_re = re.compile(r'\d{6}')

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):

    #Set mapping
    update = { "St": "Street",
            "St.": "Street",
            "Rd.": "Road",
            "Rd": "Road",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Dr": "Drive",
            "Dr.": "Drive"
            }

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        #Iterate through all attributes from node tag
        for attr in element.attrib:
            #If attributes are in FIELDS            
            if attr in node_attr_fields:
                node_attribs[attr] = element.attrib[attr]
        #Iterate through all sub-tags for element
        for sub in element:               
            #Create empty Dict
            node_elements = {}
            #Ignore if k.tag value has problematic chars
            if problem_chars.search(sub.attrib['k']):
                continue
            #If k.tag value matches lower_colon
            elif LOWER_COLON.search(sub.attrib['k']):
                node_elements['id'] = element.attrib['id']
                node_elements['key'] = sub.attrib['k'].split(':', 1)[1]
                node_elements['type'] = sub.attrib['k'].split(':', 1)[0]
                node_elements['value'] = sub.attrib['v']
                if node_elements['key'] == 'street':
                    node_elements['value'] = update_name(sub.attrib['v'], update)
                elif node_elements['key'] == 'postcode':
                    if update_pc(sub.attrib['v']):
                        node_elements['value'] = update_pc(sub.attrib['v'])
                tags.append(node_elements)
            else:
                node_elements['id'] = element.attrib['id']
                node_elements['key'] = sub.attrib['k']
                node_elements['type'] = default_tag_type
                node_elements['value'] = sub.attrib['v']
                tags.append(node_elements)
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':
        for attr in element.attrib:
            #If attributes are in FIELDS
            if attr in way_attr_fields:
                way_attribs[attr] = element.attrib[attr]
            position = 0    
        for sub in element:
            #Create empty Dict
            way_elements = {}
            way_ref = {}
            start_list = []
            if sub.tag == 'nd':
                way_ref['id'] = element.attrib['id']
                way_ref['node_id'] = sub.attrib['ref']
                way_ref['position'] = position
                way_nodes.append(way_ref)
                position += 1
            if sub.tag == 'tag':
                if problem_chars.search(sub.attrib['k']):
                    continue
                #If k.tag value matches lower_colon
                elif LOWER_COLON.search(sub.attrib['k']):
                    way_elements['id'] = element.attrib['id']
                    way_elements['key'] = sub.attrib['k'].split(':', 1)[1]
                    way_elements['type'] = sub.attrib['k'].split(':', 1)[0]
                    way_elements['value'] = sub.attrib['v']
                    if way_elements['key'] == 'street':
                        way_elements['value'] = update_name(sub.attrib['v'], update)
                    elif way_elements['key'] == 'postcode':
                        way_elements['value'] = update_pc(sub.attrib['v'])
                    tags.append(way_elements)
                else:
                    way_elements['id'] = element.attrib['id']
                    way_elements['key'] = sub.attrib['k']
                    way_elements['type'] = default_tag_type
                    way_elements['value'] = sub.attrib['v']
                    tags.append(way_elements)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# ================================================== #
#               Helper Functions                     #
# ================================================== #

#Writer function
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

#Update street names
def update_name(name, mapping):
    '''
    Args: 
        name: street value of attrib['v']
        mapping: dictionary to replace found string/character
        
    Returns:
        Updated street name'''
    for map in mapping:
        if map in name:
            name = re.sub(r'\b' + map + r'\b\.?', mapping[map], name)
    return name

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
            
#Update postal code based on continous 6-digit format in Singapore
#Searches entire string for match
def update_pc(pc):
    '''
    Args:
        pc: postcode value of attrib['v']
        
    Returns:
        Updated postcode'''
    if post_re.search(pc):
        pc = post_re.search(pc).group()
    return pc

def check_pc(osmfile):
    '''Checks if postal codes matches regular expression.
        Adds attributes which do not match to list and returns it.'''
    osm_file = open(osm_path, "r")
    pc_codes = []
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == 'addr:postcode' and not post_re.search(tag.attrib['v']):
                    pc_codes.append(tag.attrib['v'])
    osm_file.close()
    return pc_codes

# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in):
    """Iteratively process each XML element and write to csv(s)"""
    
    #Write files
    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()
        
        #Set elements
        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

def test():
   process_map(OSM_PATH)

if __name__ == '__main__':
    test()

In [52]:
import sqlite3
import csv
import pprint

#Create DB
sqlite_db = 'OSM_Project.sqlite'

#Create connection
conn = sqlite3.connect(sqlite_db)

#Create cursor
cur = conn.cursor()

#Create Tables
cur.execute('''CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);''')

cur.execute('''CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);''')

cur.execute('''CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);''')

cur.execute('''CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);''')

cur.execute('''CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);''')

#Create data for insertion
with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db_nodes = [(i['id'], i['lat'], i['lon'], i['user'].decode('utf-8'), i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
    
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db_nodes_tags = [(i['id'], i['key'].decode('utf-8'), i['value'].decode('utf-8'), i['type']) for i in dr]
    
with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db_ways = [(i['id'], i['user'].decode('utf-8'), i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
    
with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db_ways_tags = [(i['id'], i['key'], i['value'].decode('utf-8'), i['type']) for i in dr]
    
with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db_ways_nodes = [(i['id'], i['node_id'], i['position']) for i in dr]

#Insert data
cur.executemany('''Insert into nodes (id, lat, lon, user, uid, version, changeset, timestamp)
values (?,?,?,?,?,?,?,?);''', to_db_nodes)

cur.executemany('''Insert into nodes_tags (id, key, value, type)
values (?,?,?,?);''', to_db_nodes_tags)

cur.executemany('''Insert into ways (id, user, uid, version, changeset, timestamp)
values (?,?,?,?,?,?);''', to_db_ways)

cur.executemany('''Insert into ways_tags (id, key, value, type)
values (?,?,?,?);''', to_db_ways_tags)

cur.executemany('''Insert into ways_nodes (id, node_id, position)
values (?,?,?);''', to_db_ways_nodes)

#Commit changes
conn.commit()

#Close connection
conn.close()